In [ ]:
# imports
import pandas as pd
import pathlib
import json

In [ ]:
# read in raw data, downloaded direct messages from instagram, path to folder of folders where each contains the messages between the user and other users/groupchats
msgs = pathlib.Path("ig2024/messages")

In [ ]:
# declare empty dataframe
reel_data = pd.DataFrame()

In [ ]:
# format data into a dataframe
for msg in msgs.rglob("*/message_1.json"):
    with open(msg) as person:
        data = json.load(person)
        df = pd.DataFrame(data['messages'])
        if 'share' in df.columns:
            df = (df[(df['share'].apply(str).str.contains('instagram.com/reel/')) |
                  (df['share'].apply(str).str.contains('instagram.com/p/'))]
                  .assign(person = str(msg).split('/')[3])
                  [['person', 'sender_name', 'timestamp_ms']])
            reel_data = pd.concat([reel_data, df])
print(reel_data)

In [ ]:
# convert timestamp to year-month
import datetime as dt
reel_data = reel_data.assign(time = (reel_data['timestamp_ms']/1000).apply(dt.datetime.fromtimestamp)).drop(columns=['timestamp_ms'])
reel_data = reel_data.reset_index(drop=True)
timed_reels = reel_data.assign(time = [str(t)[:7] for t in reel_data['time']])

In [ ]:
# cleaned dataframe, download to device
timed_reels.to_csv('reel_data.csv')
timed_reels